# LangChain + OpenAI Context-Aware Multiplier Prediction Agent

This notebook implements a context-aware prediction agent using LangChain and OpenAI models.

**Features:**
1. **Pattern Analysis**: Agent studies multiplier patterns from CSV data
2. **Feature Engineering**: Uses all 94 features from transformer_training.ipynb
3. **Context-Aware Prediction**: Maintains historical context and patterns
4. **Real-Time Monitoring**: Waits for actual multipliers and updates predictions
5. **Prediction + Range**: Provides both predicted multiplier and confidence range

**Workflow:**
1. Load CSV and analyze patterns
2. Prepare features (94 features) + 30-50 recent multipliers
3. Send context to LLM for prediction
4. Print prediction and wait for actual
5. When actual arrives, update context and predict next


In [14]:
from dotenv import load_dotenv

load_dotenv()

False

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import time
import json
import itertools
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# LangChain and OpenAI imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

# Try to import callback for token tracking (optional)
try:
    from langchain.callbacks import get_openai_callback
    HAS_CALLBACK = True
except ImportError:
    try:
        from langchain_community.callbacks import get_openai_callback
        HAS_CALLBACK = True
    except ImportError:
        try:
            from langchain_openai.callbacks import get_openai_callback
            HAS_CALLBACK = True
        except ImportError:
            HAS_CALLBACK = False
            print("⚠️  Token tracking callbacks not available - will track tokens from response metadata")

# Set up OpenAI API Key (make sure it's set in environment)
import os
os.environ[""] = ""  # Uncomment and set your key

print("✅ Libraries imported successfully")
if not HAS_CALLBACK:
    print("   Note: Using response metadata for token tracking")


⚠️  Token tracking callbacks not available - will track tokens from response metadata
✅ Libraries imported successfully
   Note: Using response metadata for token tracking


## 1. Configuration and Data Loading


In [2]:
# Configuration
DATA_FOLDER = 'data'
CSV_FILE = os.path.join(DATA_FOLDER, 'aviator_payouts_global.csv')
LOOKBACK = 30  # Same as transformer model
RECENT_MULTIPLIERS_COUNT = 50  # Number of recent multipliers to include in context
PREDICTION_LOG_FILE = 'langchain_predictions_log.csv'

# OpenAI Model Configuration
MODEL_NAME = "gpt-5.2"  # or "gpt-4-turbo-preview" or "gpt-3.5-turbo"
TEMPERATURE = 0.3  # Lower temperature for more consistent predictions
MAX_TOKENS = 500

print(f"📁 CSV File: {CSV_FILE}")
print(f"🤖 Model: {MODEL_NAME}")
print(f"📊 Lookback: {LOOKBACK}")
print(f"📈 Recent multipliers in context: {RECENT_MULTIPLIERS_COUNT}")


📁 CSV File: data/aviator_payouts_global.csv
🤖 Model: gpt-5.2
📊 Lookback: 30
📈 Recent multipliers in context: 50


In [3]:
# Load CSV file function
def load_csv_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess CSV data."""
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV file not found: {csv_path}")
    
    df = pd.read_csv(csv_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    
    # Validate multipliers: must be >= 1.0
    df['multiplier'] = pd.to_numeric(df['multiplier'], errors='coerce')
    df = df[df['multiplier'] >= 1.0]
    df = df.dropna().reset_index(drop=True)
    
    return df

# Test load
df = load_csv_data(CSV_FILE)
print(f"✅ Loaded {len(df)} records from {CSV_FILE}")
print(f"📅 Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"📊 Multiplier statistics:")
print(df['multiplier'].describe())


✅ Loaded 8306 records from data/aviator_payouts_global.csv
📅 Date range: 2025-12-10 19:32:35 to 2025-12-13 14:55:09
📊 Multiplier statistics:
count    8306.000000
mean       10.662275
std       114.431226
min         1.000000
25%         1.290000
50%         1.920000
75%         3.790000
max      6114.720000
Name: multiplier, dtype: float64


## 2. Feature Engineering (Same as transformer_training.ipynb)


In [4]:
# Feature engineering functions (from transformer_training.ipynb)
def handle_outliers(df, multiplier_col='multiplier', method='winsorize', percentile_low=1, percentile_high=99):
    """Handle outliers in multiplier data."""
    df = df.copy()
    multipliers = df[multiplier_col].values
    if method == 'winsorize':
        low_threshold = np.percentile(multipliers, percentile_low)
        high_threshold = np.percentile(multipliers, percentile_high)
        df[multiplier_col] = df[multiplier_col].clip(lower=low_threshold, upper=high_threshold)
    return df

def prepare_features_for_prediction(df, lookback=30):
    """
    Prepare features for prediction - returns the latest feature vector.
    This is adapted from prepare_features_transformer but returns single feature vector.
    """
    if len(df) < lookback + 1:
        return None
    
    df = df.copy()
    df['multiplier'] = pd.to_numeric(df['multiplier'], errors='coerce')
    df = df.dropna().reset_index(drop=True)
    df = df.sort_values('timestamp').reset_index(drop=True)
    
    # Handle outliers
    df = handle_outliers(df, method='winsorize', percentile_low=1, percentile_high=99)
    
    if len(df) < lookback + 1:
        return None
    
    # Extract time components
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    df['second'] = df['timestamp'].dt.second
    df['time_of_day'] = df['hour'] * 3600 + df['minute'] * 60 + df['second']
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    
    # Global statistics
    global_mean = df['multiplier'].mean()
    global_std = df['multiplier'].std()
    global_median = df['multiplier'].median()
    global_min = df['multiplier'].min()
    global_max = df['multiplier'].max()
    global_p25 = df['multiplier'].quantile(0.25)
    global_p75 = df['multiplier'].quantile(0.75)
    global_p90 = df['multiplier'].quantile(0.90)
    global_p95 = df['multiplier'].quantile(0.95)
    global_skew = df['multiplier'].skew()
    global_kurtosis = df['multiplier'].kurtosis()
    
    # Pattern analysis
    high_count_global = (df['multiplier'] > 5.0).sum()
    very_high_count_global = (df['multiplier'] > 10.0).sum()
    low_count_global = (df['multiplier'] < 1.5).sum()
    extreme_count_global = (df['multiplier'] > 50.0).sum()
    
    # Long-term trend
    if len(df) > 50:
        x_indices = np.arange(len(df))
        global_trend_coef = np.polyfit(x_indices, df['multiplier'].values, 2)
        global_trend_slope = global_trend_coef[1]
        global_trend_curve = global_trend_coef[0]
    else:
        global_trend_slope = 0
        global_trend_curve = 0
    
    # Autocorrelation
    if len(df) > 100:
        autocorr_lag5 = df['multiplier'].autocorr(lag=5) if len(df) > 5 else 0
        autocorr_lag10 = df['multiplier'].autocorr(lag=10) if len(df) > 10 else 0
        autocorr_lag20 = df['multiplier'].autocorr(lag=20) if len(df) > 20 else 0
    else:
        autocorr_lag5 = autocorr_lag10 = autocorr_lag20 = 0
    
    # Transition rate
    multiplier_values = df['multiplier'].values
    if len(multiplier_values) > 50:
        high_low_transitions = sum(1 for i in range(1, len(multiplier_values)) 
                                  if (multiplier_values[i-1] < 2.0 and multiplier_values[i] > 5.0) or
                                     (multiplier_values[i-1] > 5.0 and multiplier_values[i] < 2.0))
        transition_rate = high_low_transitions / len(multiplier_values)
    else:
        transition_rate = 0
    
    # Pre-compute rolling statistics
    df['mult_rolling_mean_3'] = df['multiplier'].rolling(window=3, min_periods=1).mean()
    df['mult_rolling_mean_5'] = df['multiplier'].rolling(window=5, min_periods=1).mean()
    df['mult_rolling_mean_10'] = df['multiplier'].rolling(window=10, min_periods=1).mean()
    df['mult_rolling_mean_20'] = df['multiplier'].rolling(window=20, min_periods=1).mean()
    df['mult_rolling_std_5'] = df['multiplier'].rolling(window=5, min_periods=1).std()
    df['mult_rolling_std_10'] = df['multiplier'].rolling(window=10, min_periods=1).std()
    df['mult_rolling_std_20'] = df['multiplier'].rolling(window=20, min_periods=1).std()
    df['mult_rolling_min_5'] = df['multiplier'].rolling(window=5, min_periods=1).min()
    df['mult_rolling_max_5'] = df['multiplier'].rolling(window=5, min_periods=1).max()
    
    # Exponential moving averages
    df['mult_ema_3'] = df['multiplier'].ewm(span=3, adjust=False).mean()
    df['mult_ema_5'] = df['multiplier'].ewm(span=5, adjust=False).mean()
    df['mult_ema_10'] = df['multiplier'].ewm(span=10, adjust=False).mean()
    df['mult_ema_20'] = df['multiplier'].ewm(span=20, adjust=False).mean()
    
    # Additional features
    df['mult_diff_1'] = df['multiplier'].diff(1)
    df['mult_diff_2'] = df['multiplier'].diff(2)
    df['mult_pct_change'] = df['multiplier'].pct_change()
    
    # Get the last index for prediction
    i = len(df) - 1
    window = df['multiplier'].iloc[i-lookback:i].values
    
    # Basic statistics
    mean_val = np.mean(window)
    std_val = np.std(window) if len(window) > 1 else 0
    min_val = np.min(window)
    max_val = np.max(window)
    median_val = np.median(window)
    
    # Time features
    current_time = df['timestamp'].iloc[i]
    time_diff = (current_time - df['timestamp'].iloc[i-1]).total_seconds() if i > 0 else 0
    hour = df['hour'].iloc[i]
    minute = df['minute'].iloc[i]
    time_of_day = df['time_of_day'].iloc[i]
    day_of_week = df['day_of_week'].iloc[i]
    is_weekend = df['is_weekend'].iloc[i]
    
    # Time since last high/low
    recent_highs = [j for j in range(max(0, i-10), i) if df['multiplier'].iloc[j] > 5.0]
    time_since_high = (i - recent_highs[-1]) if recent_highs else 20
    recent_lows = [j for j in range(max(0, i-10), i) if df['multiplier'].iloc[j] < 1.5]
    time_since_low = (i - recent_lows[-1]) if recent_lows else 20
    
    # Rolling statistics (use i-1 to avoid data leakage)
    rolling_mean_3 = df['mult_rolling_mean_3'].iloc[i-1] if i > 0 else mean_val
    rolling_mean_5 = df['mult_rolling_mean_5'].iloc[i-1] if i > 0 else mean_val
    rolling_mean_10 = df['mult_rolling_mean_10'].iloc[i-1] if i > 0 else mean_val
    rolling_mean_20 = df['mult_rolling_mean_20'].iloc[i-1] if i > 0 else mean_val
    rolling_std_5 = df['mult_rolling_std_5'].iloc[i-1] if i > 0 else std_val
    rolling_std_10 = df['mult_rolling_std_10'].iloc[i-1] if i > 0 else std_val
    rolling_std_20 = df['mult_rolling_std_20'].iloc[i-1] if i > 0 else std_val
    rolling_min_5 = df['mult_rolling_min_5'].iloc[i-1] if i > 0 else min_val
    rolling_max_5 = df['mult_rolling_max_5'].iloc[i-1] if i > 0 else max_val
    
    # EMA
    ema_3 = df['mult_ema_3'].iloc[i-1] if i > 0 else mean_val
    ema_5 = df['mult_ema_5'].iloc[i-1] if i > 0 else mean_val
    ema_10 = df['mult_ema_10'].iloc[i-1] if i > 0 else mean_val
    ema_20 = df['mult_ema_20'].iloc[i-1] if i > 0 else mean_val
    
    # Sequence features
    last_1 = window[-1] if len(window) > 0 else mean_val
    last_2 = window[-2] if len(window) > 1 else last_1
    last_3 = window[-3] if len(window) > 2 else last_2
    last_5_mean = np.mean(window[-5:]) if len(window) >= 5 else mean_val
    last_10_mean = np.mean(window[-10:]) if len(window) >= 10 else mean_val
    
    # Global comparisons
    window_vs_global_mean = mean_val / global_mean if global_mean > 0 else 1
    window_vs_global_std = std_val / global_std if global_std > 0 else 1
    last_vs_global_mean = last_1 / global_mean if global_mean > 0 else 1
    last_vs_global_median = last_1 / global_median if global_median > 0 else 1
    percentile_in_global = (df['multiplier'].iloc[:i+1] < last_1).sum() / (i + 1) if i > 0 else 0.5
    dist_from_global_mean = last_1 - global_mean
    dist_from_global_median = last_1 - global_median
    z_score_global = (last_1 - global_mean) / global_std if global_std > 0 else 0
    
    # Trends
    recent_5 = window[-5:] if len(window) >= 5 else window
    recent_10 = window[-10:] if len(window) >= 10 else window
    trend_5 = np.polyfit(range(len(recent_5)), recent_5, 1)[0] if len(recent_5) > 1 else 0
    trend_10 = np.polyfit(range(len(recent_10)), recent_10, 1)[0] if len(recent_10) > 1 else 0
    
    # Volatility
    volatility_5 = np.std(window[-5:]) if len(window) >= 5 else std_val
    volatility_10 = np.std(window[-10:]) if len(window) >= 10 else std_val
    
    # Patterns
    high_ratio = len([x for x in window if x > 2.0]) / len(window)
    very_high_ratio = len([x for x in window if x > 10.0]) / len(window)
    low_ratio = len([x for x in window if x < 1.5]) / len(window)
    extreme_high_ratio = len([x for x in window if x > 50.0]) / len(window)
    
    # Momentum
    momentum_1 = last_1 - last_2 if len(window) > 1 else 0
    momentum_2 = last_2 - last_3 if len(window) > 2 else 0
    acceleration = momentum_1 - momentum_2
    
    # Rate of change
    roc_1 = (last_1 - last_2) / last_2 if len(window) > 1 and last_2 > 0 else 0
    roc_2 = (last_2 - last_3) / last_3 if len(window) > 2 and last_3 > 0 else 0
    roc_5 = (last_1 - window[-6]) / window[-6] if len(window) > 5 and window[-6] > 0 else 0
    
    # Autocorrelation
    autocorr_1 = np.corrcoef(window[:-1], window[1:])[0, 1] if len(window) > 1 and not np.isnan(np.corrcoef(window[:-1], window[1:])[0, 1]) else 0
    
    # Range features
    range_val = max_val - min_val
    range_ratio = range_val / mean_val if mean_val > 0 else 0
    cv = std_val / mean_val if mean_val > 0 else 0
    
    # Percentiles
    p10 = np.percentile(window, 10)
    p25 = np.percentile(window, 25)
    p50 = np.percentile(window, 50)
    p75 = np.percentile(window, 75)
    p90 = np.percentile(window, 90)
    iqr = p75 - p25
    
    # MA ratios
    ma_ratio_3_5 = rolling_mean_3 / rolling_mean_5 if rolling_mean_5 > 0 else 1
    ma_ratio_5_10 = rolling_mean_5 / rolling_mean_10 if rolling_mean_10 > 0 else 1
    ema_ratio_3_5 = ema_3 / ema_5 if ema_5 > 0 else 1
    
    # Distance from MA
    dist_from_ma3 = last_1 - rolling_mean_3
    dist_from_ma5 = last_1 - rolling_mean_5
    dist_from_ma10 = last_1 - rolling_mean_10
    dist_from_ema3 = last_1 - ema_3
    
    # Z-scores
    z_score_5 = (last_1 - rolling_mean_5) / rolling_std_5 if rolling_std_5 > 0 else 0
    z_score_10 = (last_1 - rolling_mean_10) / rolling_std_10 if rolling_std_10 > 0 else 0
    
    # Additional features
    if i > 0:
        diff_1 = df['mult_diff_1'].iloc[i-1] if not pd.isna(df['mult_diff_1'].iloc[i-1]) else 0
    else:
        diff_1 = 0
    if i > 1:
        diff_2 = df['mult_diff_2'].iloc[i-1] if not pd.isna(df['mult_diff_2'].iloc[i-1]) else 0
    else:
        diff_2 = 0
    if i > 0:
        pct_change = df['mult_pct_change'].iloc[i-1] if not pd.isna(df['mult_pct_change'].iloc[i-1]) else 0
    else:
        pct_change = 0
    
    # Create feature dictionary for better readability
    features_dict = {
        # Global features (20)
        'global_mean': global_mean, 'global_std': global_std, 'global_median': global_median,
        'global_min': global_min, 'global_max': global_max,
        'global_p25': global_p25, 'global_p75': global_p75, 'global_p90': global_p90, 'global_p95': global_p95,
        'global_skew': global_skew, 'global_kurtosis': global_kurtosis,
        'high_count_ratio': high_count_global / len(df), 'very_high_count_ratio': very_high_count_global / len(df),
        'low_count_ratio': low_count_global / len(df), 'extreme_count_ratio': extreme_count_global / len(df),
        'global_trend_slope': global_trend_slope, 'global_trend_curve': global_trend_curve,
        'autocorr_lag5': autocorr_lag5, 'autocorr_lag10': autocorr_lag10, 'autocorr_lag20': autocorr_lag20,
        'transition_rate': transition_rate,
        # Local window features (60+)
        'window_mean': mean_val, 'window_std': std_val, 'window_min': min_val, 'window_max': max_val, 'window_median': median_val,
        'last_1': last_1, 'last_2': last_2, 'last_3': last_3,
        'last_5_mean': last_5_mean, 'last_10_mean': last_10_mean,
        'rolling_mean_3': rolling_mean_3, 'rolling_mean_5': rolling_mean_5, 'rolling_mean_10': rolling_mean_10, 'rolling_mean_20': rolling_mean_20,
        'rolling_std_5': rolling_std_5, 'rolling_std_10': rolling_std_10, 'rolling_std_20': rolling_std_20,
        'rolling_min_5': rolling_min_5, 'rolling_max_5': rolling_max_5,
        'ema_3': ema_3, 'ema_5': ema_5, 'ema_10': ema_10, 'ema_20': ema_20,
        'trend_5': trend_5, 'trend_10': trend_10,
        'volatility_5': volatility_5, 'volatility_10': volatility_10,
        'high_ratio': high_ratio, 'very_high_ratio': very_high_ratio, 'low_ratio': low_ratio, 'extreme_high_ratio': extreme_high_ratio,
        'momentum_1': momentum_1, 'momentum_2': momentum_2, 'acceleration': acceleration,
        'roc_1': roc_1, 'roc_2': roc_2, 'roc_5': roc_5,
        'range_val': range_val, 'range_ratio': range_ratio, 'cv': cv,
        'p10': p10, 'p25': p25, 'p50': p50, 'p75': p75, 'p90': p90, 'iqr': iqr,
        'ma_ratio_3_5': ma_ratio_3_5, 'ma_ratio_5_10': ma_ratio_5_10, 'ema_ratio_3_5': ema_ratio_3_5,
        'dist_from_ma3': dist_from_ma3, 'dist_from_ma5': dist_from_ma5, 'dist_from_ma10': dist_from_ma10, 'dist_from_ema3': dist_from_ema3,
        'z_score_5': z_score_5, 'z_score_10': z_score_10,
        # Comparison features (8)
        'window_vs_global_mean': window_vs_global_mean, 'window_vs_global_std': window_vs_global_std,
        'last_vs_global_mean': last_vs_global_mean, 'last_vs_global_median': last_vs_global_median,
        'percentile_in_global': percentile_in_global,
        'dist_from_global_mean': dist_from_global_mean, 'dist_from_global_median': dist_from_global_median,
        'z_score_global': z_score_global,
        # Time features (7)
        'time_diff': time_diff, 'hour': hour, 'minute': minute, 'time_of_day': time_of_day,
        'time_since_high': time_since_high, 'day_of_week': day_of_week, 'is_weekend': is_weekend,
        # Additional features (3)
        'diff_1': diff_1, 'diff_2': diff_2, 'pct_change': pct_change,
        # Autocorrelation (1)
        'autocorr_1': autocorr_1,
    }
    
    return features_dict

print("✅ Feature engineering functions loaded")


✅ Feature engineering functions loaded


In [5]:
# Initialize LangChain Chat Model
# Note: langchain-openai uses 'model' parameter (not 'model_name')
llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS
)

print(f"✅ LangChain model initialized: {MODEL_NAME}")


✅ LangChain model initialized: gpt-5.2


In [6]:
# Create system prompt template
SYSTEM_PROMPT = """You are an expert data analyst specializing in time series prediction, specifically for multiplier patterns in gaming/financial data.

Your task is to:
1. Analyze the provided features and recent multiplier history
2. Identify patterns, trends, and statistical indicators
3. Predict the NEXT multiplier value
4. Provide a confidence range (min, max) for your prediction

IMPORTANT GUIDELINES:
- Multipliers are typically >= 1.0
- The data shows high volatility with both low multipliers (1.0-2.0) and high multipliers (5.0-100+)
- Consider: trends, volatility, recent patterns, momentum, and statistical indicators
- Provide predictions in JSON format: {{"prediction": <value>, "range_min": <value>, "range_max": <value>, "confidence": "high/medium/low", "reasoning": "<brief explanation>"}}

Be precise and data-driven in your analysis."""

# Create prediction prompt template
PREDICTION_PROMPT = """Analyze the following data and predict the next multiplier:

**Recent Multipliers (last {recent_count}):**
{recent_multipliers}

**Key Statistical Features:**
{features_summary}

**Recent Pattern Analysis:**
- Last 3 multipliers: {last_3}
- Last 10 average: {last_10_avg:.2f}
- Recent trend: {trend_description}
- Volatility (last 10): {volatility:.2f}
- Current momentum: {momentum_description}

**Historical Context:**
{context_summary}

Based on this analysis, predict the NEXT multiplier value and provide a confidence range.

Respond ONLY with valid JSON in this exact format:
{{
    "prediction": <float>,
    "range_min": <float>,
    "range_max": <float>,
    "confidence": "<high|medium|low>",
    "reasoning": "<brief explanation of your analysis>"
}}"""

def format_features_for_llm(features_dict: Dict) -> str:
    """Format features dictionary into readable string for LLM."""
    summary = f"""Global Statistics:
- Mean: {features_dict['global_mean']:.2f}, Std: {features_dict['global_std']:.2f}, Median: {features_dict['global_median']:.2f}
- Min: {features_dict['global_min']:.2f}, Max: {features_dict['global_max']:.2f}

Window Statistics (last {LOOKBACK}):
- Mean: {features_dict['window_mean']:.2f}, Std: {features_dict['window_std']:.2f}
- Min: {features_dict['window_min']:.2f}, Max: {features_dict['window_max']:.2f}

Trends & Momentum:
- Short-term trend (5): {features_dict['trend_5']:.4f}
- Medium-term trend (10): {features_dict['trend_10']:.4f}
- Momentum: {features_dict['momentum_1']:.4f}, Acceleration: {features_dict['acceleration']:.4f}
- Rate of change (1): {features_dict['roc_1']:.4f}

Moving Averages:
- Rolling MA(5): {features_dict['rolling_mean_5']:.2f}, MA(10): {features_dict['rolling_mean_10']:.2f}
- EMA(5): {features_dict['ema_5']:.2f}, EMA(10): {features_dict['ema_10']:.2f}

Pattern Indicators:
- High ratio (>2.0): {features_dict['high_ratio']:.2%}
- Very high ratio (>10.0): {features_dict['very_high_ratio']:.2%}
- Low ratio (<1.5): {features_dict['low_ratio']:.2%}

Volatility:
- Recent (5): {features_dict['volatility_5']:.2f}, Medium (10): {features_dict['volatility_10']:.2f}
- Coefficient of variation: {features_dict['cv']:.4f}

Position vs Global:
- Last value vs global mean: {features_dict['last_vs_global_mean']:.2f}x
- Z-score (global): {features_dict['z_score_global']:.2f}
- Percentile in global: {features_dict['percentile_in_global']:.2%}

Time Features:
- Hour: {int(features_dict['hour'])}, Day of week: {int(features_dict['day_of_week'])}
- Time since last high: {int(features_dict['time_since_high'])}
"""
    return summary

def format_recent_multipliers(multipliers: List[float], count: int = 50) -> str:
    """Format recent multipliers for LLM context."""
    recent = multipliers[-count:] if len(multipliers) > count else multipliers
    formatted = []
    for i, mult in enumerate(recent):
        formatted.append(f"  {i+1}. {mult:.2f}x")
    return "\\n".join(formatted)

print("✅ Prompt templates created")


✅ Prompt templates created


In [7]:
def analyze_patterns_and_predict(
    df: pd.DataFrame,
    context_history: List[Dict] = None,
    recent_count: int = RECENT_MULTIPLIERS_COUNT
) -> Dict:
    """
    Analyze patterns and get prediction from LLM.
    
    Args:
        df: DataFrame with multiplier data
        context_history: List of previous predictions and actuals for context
        recent_count: Number of recent multipliers to include
    
    Returns:
        Dictionary with prediction, range, and metadata
    """
    # Prepare features
    features_dict = prepare_features_for_prediction(df, lookback=LOOKBACK)
    if features_dict is None:
        return None
    
    # Get recent multipliers
    recent_multipliers = df['multiplier'].tail(recent_count).tolist()
    last_3 = recent_multipliers[-3:]
    last_10_avg = np.mean(recent_multipliers[-10:]) if len(recent_multipliers) >= 10 else np.mean(recent_multipliers)
    
    # Trend description
    if len(recent_multipliers) >= 5:
        recent_5 = recent_multipliers[-5:]
        trend_slope = np.polyfit(range(len(recent_5)), recent_5, 1)[0]
        if trend_slope > 0.1:
            trend_desc = "Increasing trend"
        elif trend_slope < -0.1:
            trend_desc = "Decreasing trend"
        else:
            trend_desc = "Stable/Flat trend"
    else:
        trend_desc = "Insufficient data"
    
    # Momentum description
    momentum_1 = features_dict['momentum_1']
    if momentum_1 > 0.5:
        momentum_desc = f"Strong upward momentum ({momentum_1:.2f})"
    elif momentum_1 < -0.5:
        momentum_desc = f"Strong downward momentum ({momentum_1:.2f})"
    else:
        momentum_desc = f"Neutral momentum ({momentum_1:.2f})"
    
    # Context summary
    if context_history and len(context_history) > 0:
        recent_context = context_history[-5:]  # Last 5 predictions
        context_summary = "Recent prediction accuracy:\\n"
        for ctx in recent_context:
            if 'actual' in ctx and ctx['actual'] is not None:
                error = abs(ctx['actual'] - ctx['prediction'])
                context_summary += f"- Predicted: {ctx['prediction']:.2f}, Actual: {ctx['actual']:.2f}, Error: {error:.2f}\\n"
    else:
        context_summary = "This is the first prediction."
    
    # Format features
    features_summary = format_features_for_llm(features_dict)
    recent_multipliers_str = format_recent_multipliers(recent_multipliers, recent_count)
    
    # Build prompt
    prompt = PREDICTION_PROMPT.format(
        recent_count=len(recent_multipliers),
        recent_multipliers=recent_multipliers_str,
        features_summary=features_summary,
        last_3=last_3,
        last_10_avg=last_10_avg,
        trend_description=trend_desc,
        volatility=features_dict['volatility_10'],
        momentum_description=momentum_desc,
        context_summary=context_summary
    )
    
    # Get prediction from LLM
    messages = [
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content=prompt)
    ]
    
    try:
        # Get prediction from LLM with optional token tracking
        tokens_used = 0
        cost = 0.0
        
        if HAS_CALLBACK:
            # Use callback if available
            with get_openai_callback() as cb:
                response = llm.invoke(messages)
                tokens_used = cb.total_tokens if hasattr(cb, 'total_tokens') else 0
                cost = cb.total_cost if hasattr(cb, 'total_cost') else 0.0
        else:
            # Call without callback and try to get token info from response
            response = llm.invoke(messages)
            # Try to extract token usage from response metadata
            if hasattr(response, 'response_metadata'):
                metadata = response.response_metadata
                if 'token_usage' in metadata:
                    token_usage = metadata['token_usage']
                    tokens_used = token_usage.get('total_tokens', 0)
                    # Estimate cost (approximate, model-specific pricing)
                    # GPT-4o pricing (as of 2024): $5.00/1M input tokens, $15.00/1M output tokens
                    input_tokens = token_usage.get('prompt_tokens', 0)
                    output_tokens = token_usage.get('completion_tokens', 0)
                    cost = (input_tokens / 1_000_000 * 5.0) + (output_tokens / 1_000_000 * 15.0)
        
        prediction_text = response.content.strip()
        
        # Parse JSON response
        # Try to extract JSON from response
        if "```json" in prediction_text:
            prediction_text = prediction_text.split("```json")[1].split("```")[0].strip()
        elif "```" in prediction_text:
            prediction_text = prediction_text.split("```")[1].split("```")[0].strip()
        
        # Try to parse as JSON
        try:
            prediction_data = json.loads(prediction_text)
        except json.JSONDecodeError:
            # Try to extract values using regex if JSON parsing fails
            import re
            pred_match = re.search(r'"prediction"\s*:\s*([0-9.]+)', prediction_text)
            min_match = re.search(r'"range_min"\s*:\s*([0-9.]+)', prediction_text)
            max_match = re.search(r'"range_max"\s*:\s*([0-9.]+)', prediction_text)
            conf_match = re.search(r'"confidence"\s*:\s*"([^"]+)"', prediction_text)
            
            if pred_match and min_match and max_match:
                prediction_data = {
                    'prediction': float(pred_match.group(1)),
                    'range_min': float(min_match.group(1)),
                    'range_max': float(max_match.group(1)),
                    'confidence': conf_match.group(1) if conf_match else "medium",
                    'reasoning': "Extracted from LLM response"
                }
            else:
                raise ValueError("Could not parse prediction from LLM response")
        
        # Validate prediction
        prediction_data['prediction'] = max(1.0, float(prediction_data['prediction']))
        prediction_data['range_min'] = max(1.0, float(prediction_data['range_min']))
        prediction_data['range_max'] = max(1.0, float(prediction_data['range_max']))
        
        # Add metadata
        prediction_data['timestamp'] = datetime.now().isoformat()
        prediction_data['tokens_used'] = tokens_used
        prediction_data['cost'] = cost
        
        return prediction_data
            
    except Exception as e:
        print(f"❌ Error getting prediction from LLM: {e}")
        return None

print("✅ Prediction function created")


✅ Prediction function created


## 4. Initial Pattern Study


In [8]:
def study_patterns(df: pd.DataFrame) -> Dict:
    """Initial study of patterns in the data."""
    print("🔍 Studying multiplier patterns...")
    
    patterns = {
        'total_samples': len(df),
        'mean': df['multiplier'].mean(),
        'median': df['multiplier'].median(),
        'std': df['multiplier'].std(),
        'min': df['multiplier'].min(),
        'max': df['multiplier'].max(),
        'p25': df['multiplier'].quantile(0.25),
        'p75': df['multiplier'].quantile(0.75),
        'p90': df['multiplier'].quantile(0.90),
        'high_count': (df['multiplier'] > 5.0).sum(),
        'very_high_count': (df['multiplier'] > 10.0).sum(),
        'low_count': (df['multiplier'] < 1.5).sum(),
        'extreme_count': (df['multiplier'] > 50.0).sum(),
        'date_range': {
            'start': df['timestamp'].min().isoformat(),
            'end': df['timestamp'].max().isoformat()
        }
    }
    
    # Analyze sequences
    multipliers = df['multiplier'].values
    low_groups = [sum(1 for _ in group) for key, group in itertools.groupby(multipliers < 1.5) if key]
    high_groups = [sum(1 for _ in group) for key, group in itertools.groupby(multipliers > 5.0) if key]
    patterns['avg_consecutive_lows'] = np.mean(low_groups) if low_groups else 0
    patterns['avg_consecutive_highs'] = np.mean(high_groups) if high_groups else 0
    
    print(f"✅ Pattern study complete:")
    print(f"   Total samples: {patterns['total_samples']}")
    print(f"   Mean: {patterns['mean']:.2f}, Median: {patterns['median']:.2f}")
    print(f"   High multipliers (>5.0): {patterns['high_count']} ({patterns['high_count']/len(df)*100:.1f}%)")
    print(f"   Low multipliers (<1.5): {patterns['low_count']} ({patterns['low_count']/len(df)*100:.1f}%)")
    
    return patterns

# Study initial patterns
initial_patterns = study_patterns(df)
print("\\n📊 Initial patterns stored for context")


🔍 Studying multiplier patterns...
✅ Pattern study complete:
   Total samples: 8306
   Mean: 10.66, Median: 1.92
   High multipliers (>5.0): 1599 (19.3%)
   Low multipliers (<1.5): 2950 (35.5%)
\n📊 Initial patterns stored for context


## 5. Prediction Monitoring Loop


In [9]:
# Initialize prediction log
prediction_log = []
context_history = []
pending_prediction_index = None  # Index we're currently predicting for
pending_prediction_data = None   # The prediction data we're waiting for

def save_prediction_log():
    """Save prediction log to CSV."""
    if len(prediction_log) == 0:
        return
    
    log_df = pd.DataFrame(prediction_log)
    log_df.to_csv(PREDICTION_LOG_FILE, index=False)
    print(f"💾 Prediction log saved to {PREDICTION_LOG_FILE}")

def monitor_and_predict(check_interval: float = 2.0, max_iterations: Optional[int] = None):
    """
    Monitor CSV for new data and make predictions using LangChain agent.
    
    Proper workflow:
    1. Read CSV in loop
    2. On new data:
       - If no pending prediction -> predict for next index, store index
       - If pending prediction -> check stored index, print actual, then predict next
    
    Args:
        check_interval: Seconds between CSV checks
        max_iterations: Maximum number of iterations (None for infinite)
    """
    global df, prediction_log, context_history, pending_prediction_index, pending_prediction_data
    
    # Start from the last row we have data for
    last_known_row = len(df) - 1
    iteration = 0
    
    print("=" * 80)
    print("🚀 Starting LangChain Prediction Agent")
    print("=" * 80)
    print(f"📁 Monitoring: {CSV_FILE}")
    print(f"⏱️  Check interval: {check_interval} seconds")
    print(f"🔄 Starting from row {last_known_row} (last known index: {last_known_row})")
    print("=" * 80)
    print("\\nPress Ctrl+C to stop\\n")
    
    try:
        while True:
            if max_iterations and iteration >= max_iterations:
                break
            
            # Load current CSV
            try:
                current_df = load_csv_data(CSV_FILE)
            except Exception as e:
                print(f"⚠️  Error loading CSV: {e}")
                time.sleep(check_interval)
                continue
            
            current_row_count = len(current_df)
            
            # Check if we have new data (file has grown)
            if current_row_count > last_known_row + 1:
                # New data available!
                new_rows_count = current_row_count - (last_known_row + 1)
                print(f"\\n📈 New data detected: {new_rows_count} new row(s) (Total: {current_row_count}, Last known: {last_known_row})")
                
                # Update df
                df = current_df
                
                # Case 1: We have a pending prediction - check if actual is now available
                if pending_prediction_index is not None:
                    # Check if the actual value is available at the stored index
                    if pending_prediction_index < len(df):
                        # Actual value is now available!
                        actual_value = float(df['multiplier'].iloc[pending_prediction_index])
                        actual_timestamp = df['timestamp'].iloc[pending_prediction_index]
                        
                        # Get the prediction data
                        pred_value = pending_prediction_data['prediction']
                        pred_min = pending_prediction_data['range_min']
                        pred_max = pending_prediction_data['range_max']
                        
                        # Calculate errors
                        error = actual_value - pred_value
                        abs_error = abs(error)
                        pct_error = abs(error / actual_value * 100) if actual_value > 0 else 0
                        in_range = (pred_min <= actual_value <= pred_max)
                        
                        # Print actual value
                        print("\\n" + "=" * 80)
                        print("✅ ACTUAL MULTIPLIER")
                        print("=" * 80)
                        print(f"📊 Index: {pending_prediction_index}")
                        print(f"📊 Actual Multiplier: {actual_value:.4f}x")
                        print(f"🔮 Predicted: {pred_value:.4f}x")
                        print(f"📏 Range: {pred_min:.4f}x - {pred_max:.4f}x")
                        print(f"❌ Error: {error:+.4f} ({pct_error:.2f}%)")
                        print(f"{'✅' if in_range else '❌'} In Range: {in_range}")
                        print(f"⏰ Timestamp: {actual_timestamp}")
                        print("=" * 80)
                        
                        # Update the pending log entry
                        if len(prediction_log) > 0:
                            log_entry = prediction_log[-1]
                            log_entry['actual_value'] = actual_value
                            log_entry['actual_timestamp'] = actual_timestamp.isoformat() if hasattr(actual_timestamp, 'isoformat') else str(actual_timestamp)
                            log_entry['error'] = error
                            log_entry['abs_error'] = abs_error
                            log_entry['pct_error'] = pct_error
                            log_entry['in_range'] = in_range
                            log_entry['status'] = 'completed'
                        
                        # Update context history
                        if len(context_history) > 0:
                            context_history[-1]['actual'] = actual_value
                        
                        # Save log
                        save_prediction_log()
                        
                        # Store the index we just processed before clearing
                        processed_index = pending_prediction_index
                        
                        # Clear pending prediction
                        pending_prediction_index = None
                        pending_prediction_data = None
                        
                        # Update last_known_row to the index we just processed
                        last_known_row = processed_index
                
                # Case 2: No pending prediction OR we just processed one -> make new prediction
                if pending_prediction_index is None:
                    # We need to predict for the next index (last_known_row + 1)
                    next_prediction_index = last_known_row + 1
                    
                    # Check if we have enough data to make a prediction
                    if len(df) < LOOKBACK + 1:
                        print(f"⚠️  Insufficient data for prediction (need {LOOKBACK + 1}, have {len(df)})")
                        last_known_row = current_row_count - 1
                        time.sleep(check_interval)
                        iteration += 1
                        continue
                    
                    # Make prediction for next index
                    print(f"🔮 Making prediction for index {next_prediction_index}...")
                    prediction_result = analyze_patterns_and_predict(df, context_history=context_history)
                    
                    if prediction_result:
                        pred_value = prediction_result['prediction']
                        pred_min = prediction_result['range_min']
                        pred_max = prediction_result['range_max']
                        confidence = prediction_result.get('confidence', 'medium')
                        reasoning = prediction_result.get('reasoning', '')
                        
                        # Print prediction FIRST
                        print("\\n" + "=" * 80)
                        print("🔮 PREDICTION")
                        print("=" * 80)
                        print(f"📊 Predicting for index: {next_prediction_index}")
                        print(f"📊 Predicted Multiplier: {pred_value:.4f}x")
                        print(f"📏 Range: {pred_min:.4f}x - {pred_max:.4f}x")
                        print(f"🎯 Confidence: {confidence.upper()}")
                        if reasoning:
                            print(f"💭 Reasoning: {reasoning}")
                        print(f"⏰ Timestamp: {prediction_result['timestamp']}")
                        if 'tokens_used' in prediction_result:
                            print(f"🔤 Tokens used: {prediction_result['tokens_used']}")
                        print("=" * 80)
                        print(f"\\n⏳ Waiting for actual multiplier at index {next_prediction_index}...")
                        
                        # Save prediction (without actual yet)
                        log_entry = {
                            'prediction_index': next_prediction_index,
                            'prediction_timestamp': prediction_result['timestamp'],
                            'predicted_value': pred_value,
                            'range_min': pred_min,
                            'range_max': pred_max,
                            'confidence': confidence,
                            'actual_value': None,
                            'actual_timestamp': None,
                            'error': None,
                            'abs_error': None,
                            'pct_error': None,
                            'in_range': None,
                            'status': 'pending',
                            'tokens_used': prediction_result.get('tokens_used', 0),
                            'cost': prediction_result.get('cost', 0)
                        }
                        
                        prediction_log.append(log_entry)
                        context_history.append({
                            'prediction': pred_value,
                            'range_min': pred_min,
                            'range_max': pred_max,
                            'timestamp': prediction_result['timestamp']
                        })
                        
                        # Store pending prediction
                        pending_prediction_index = next_prediction_index
                        pending_prediction_data = {
                            'prediction': pred_value,
                            'range_min': pred_min,
                            'range_max': pred_max,
                            'confidence': confidence,
                            'reasoning': reasoning,
                            'timestamp': prediction_result['timestamp']
                        }
                        
                        # Don't update last_known_row yet - we'll update it when we get the actual
                        # last_known_row stays at the last row we've seen data for
                    else:
                        print("⚠️  Could not get prediction from LLM agent")
                        last_known_row = current_row_count - 1
            else:
                # No new data, just wait
                time.sleep(check_interval)
            
            iteration += 1
            
    except KeyboardInterrupt:
        print("\\n\\n⏹️  Monitoring stopped by user")
        print("\\n📊 Final Statistics:")
        if len(prediction_log) > 0:
            completed = [p for p in prediction_log if p['status'] == 'completed']
            if len(completed) > 0:
                completed_df = pd.DataFrame(completed)
                print(f"   Total predictions: {len(prediction_log)}")
                print(f"   Completed: {len(completed)}")
                print(f"   Pending: {len(prediction_log) - len(completed)}")
                print(f"   Average absolute error: {completed_df['abs_error'].mean():.4f}")
                print(f"   Average percentage error: {completed_df['pct_error'].mean():.2f}%")
                print(f"   Range coverage: {(completed_df['in_range'].sum() / len(completed) * 100):.1f}%")
                
                total_tokens = completed_df['tokens_used'].sum() if 'tokens_used' in completed_df.columns else 0
                total_cost = completed_df['cost'].sum() if 'cost' in completed_df.columns else 0
                print(f"   Total tokens used: {total_tokens}")
                print(f"   Total cost: ${total_cost:.4f}")
        
        save_prediction_log()
        print(f"\\n✅ Prediction log saved to: {PREDICTION_LOG_FILE}")

print("✅ Monitoring function ready")
print("\\nTo start monitoring, run:")
print("monitor_and_predict(check_interval=2.0)")
print("\\nOr with limited iterations:")
print("monitor_and_predict(check_interval=2.0, max_iterations=10)")


✅ Monitoring function ready
\nTo start monitoring, run:
monitor_and_predict(check_interval=2.0)
\nOr with limited iterations:
monitor_and_predict(check_interval=2.0, max_iterations=10)


In [10]:
# Test with current data
print("🧪 Testing LangChain agent with single prediction...")
test_prediction = analyze_patterns_and_predict(df, context_history=[])

if test_prediction:
    print("\\n✅ Test prediction successful!")
    print(f"   Prediction: {test_prediction['prediction']:.4f}x")
    print(f"   Range: {test_prediction['range_min']:.4f}x - {test_prediction['range_max']:.4f}x")
    print(f"   Confidence: {test_prediction.get('confidence', 'N/A')}")
    if 'reasoning' in test_prediction:
        print(f"   Reasoning: {test_prediction['reasoning']}")
else:
    print("❌ Test prediction failed")


🧪 Testing LangChain agent with single prediction...
\n✅ Test prediction successful!
   Prediction: 1.6200x
   Range: 1.0000x - 4.2000x
   Confidence: low
   Reasoning: The last value (142.96x) is an extreme outlier relative to the global/rolling distributions (median ~1.92, high volatility, heavy right tail). After very high spikes, the series typically mean-reverts toward the low/typical band (1.0–2.5). Despite short-term upward momentum being inflated by the spike, the medium-term trend is weak and acceleration is negative, supporting a pullback. Prediction centers near the common low-mode with a wide range due to volatility.


## 7. Start Monitoring

Run the cell below to start monitoring the CSV file and making real-time predictions:


In [11]:
# Start monitoring
# Uncomment the line below to start:
monitor_and_predict(check_interval=1)

# Or use limited iterations for testing:
# monitor_and_predict(check_interval=2.0, max_iterations=5)

print("Ready to start monitoring. Uncomment the monitor_and_predict() call above.")


🚀 Starting LangChain Prediction Agent
📁 Monitoring: data/aviator_payouts_global.csv
⏱️  Check interval: 1 seconds
🔄 Starting from row 8305 (last known index: 8305)
\nPress Ctrl+C to stop\n
\n📈 New data detected: 2 new row(s) (Total: 8308, Last known: 8305)
🔮 Making prediction for index 8306...
\n================================================================================
🔮 PREDICTION
📊 Predicting for index: 8306
📊 Predicted Multiplier: 1.8600x
📏 Range: 1.0200x - 4.9000x
🎯 Confidence: LOW
💭 Reasoning: After an extreme spike (142.96) the series shows strong mean-reversion and sharp negative momentum/acceleration with a continuing decreasing trend. Post-spike sequences in this history frequently revert to low/near-median outcomes (median ~1.92; substantial mass in 1.0–2.0), so the next value is biased toward a low multiplier, while high recent volatility keeps the upper range wide.
⏰ Timestamp: 2025-12-13T14:56:10.670441
🔤 Tokens used: 1341
\n⏳ Waiting for actual multiplier at index 8